## <center> ENSF 519.01 Programming Fundamentals for Data Engineers </center>
<h2> <center> Assignment 7: Web data mining (100 marks)</center></h2>
<h3> <center> Due: by  Nov 9th, 04.59 pm. To be submitted on D2L. </center></h3>
<!--h3> <center> <font color='red'>IMPORTANT NOTE: each task must be implemented exactly as asked, even if there are other easier or better solutions.</font> </center></h3-->




Edit this file and write your solutions to the problems in sections specified with `# Your solution goes here`. Test your code and when you were done, download this notebook as an `.ipynb` file and submit it to D2L. To get this file, in Jupyter notebook you can go to File -> Download as -> Notebook(.ipynb)

## Part 1: Billboard top charts

Complete the functions below to get the best selling albums of a certain week from the [billboard 200](https://www.billboard.com/charts/billboard-200) page. Create a pandas dataframe to store the information. Your dataframe will have (at least) three columns:

* title (the album name)
* artist
* rank

Feel free to extract more information (like the number of weeks that it has been on the chart, a url to the album cover art, ...) if available and you're interested.


In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

from typing import Iterable

def get_billboard_top_albums_dataframe(date: str='2001-06-02', count: int=5) -> pd.DataFrame:
    Url = 'https://www.billboard.com/charts/billboard-200/' + date
    result = requests.get(Url)
    c= result.text
    soup = BeautifulSoup(c,'html.parser')
    firstTitle = soup.find('div',attrs={'class':'chart-number-one__title'}).text.strip()
    firstArtist= soup.find('div',attrs={'class':'chart-number-one__artist'}).text.strip()
    Frame = pd.DataFrame(columns = ["Title","Artist","Rank"])
    Frame2 = Frame.append({'Title':firstTitle,'Artist':firstArtist,'Rank':1},ignore_index=True)
    summary4=soup.find_all('div',attrs={'class':'chart-list-item'},limit=count-1)
    for row in summary4:
        title=row.find('div',attrs={'class':'chart-list-item__title'}).text.strip()
        artist=row.find('div',attrs ={'class':'chart-list-item__artist'}).text.strip()
        rank=row.find('div',attrs={'class':'chart-list-item__rank'}).text.strip()
        Frame2=Frame2.append({'Title':title, 'Artist':artist,'Rank':rank},ignore_index=True)
    return(Frame2)

    
    # Your solution goes here
    # Feel free to add auxilary functions, more imports, etc 
     # delete this "pass" statement please.

top_5_albums = get_billboard_top_albums_dataframe(count=5, date='2018-06-02')
top_5_albums


,Title,Artist,Rank
0,Love Yourself: Tear,BTS,1
1,beerbongs & bentleys,Post Malone,2
2,Harder Than Ever,Lil Baby,3
3,And Justice For None,Five Finger Death Punch,4
4,Invasion Of Privacy,Cardi B,5


## Music brainz API
Here's a code that connects to the music brainz music database API and searches for the information about the given album. 

In [2]:
import itertools
import unicodedata
import re

_remove_accents = lambda input_str: ''.join((c for c in unicodedata.normalize('NFKD', input_str) if not unicodedata.combining(c)))
_clean_string = lambda s: set(re.sub(r'[^\w\s]', '', _remove_accents(s)).lower().split())
_jaccard = lambda set1, set2: float(len(set1 & set2)) / float(len(set1 | set2))


def search(entity_type:str, query: str):
    return requests.get(
        'http://musicbrainz.org/ws/2/{entity}/'.format(entity=entity_type),
        params={
            'fmt': 'json',
            'query': query
        }
    ).json()


def get_release_url(artist: str, title: str):
    type_ = 'release'
    search_results = search(type_, '%s AND artist:%s' % (title, artist))
    
    artist = _clean_string(artist)
    title = _clean_string(title)

#     print("title = " + str(title) +' artist=' + str(artist))
    for item in search_results.get(type_+'s', []):
        names = list()
        for artists in item['artist-credit']:
            if 'artist' in artists:
                names.append(_clean_string(artists['artist']['name']))
                for alias in artists['artist'].get('aliases', {}):
                    names.append(_clean_string(alias.get('name', '')))
#         print('  title=' + str(_clean_string(item['title'])) + ' names=' + ', '.join(itertools.chain(*names)))

        if _jaccard(_clean_string(item['title']), title) > 0.5 and \
            (any(_jaccard(artist, name) > 0.3 for name in names) or len(names) == 0):
            return 'http://musicbrainz.org/ws/2/{type}/{id}/'.format(id=item['id'], type=type_)
    
    return None


Using the above function find the track count and disk count of each album (if available) and store it in the pandas data frame. Read the implementation of the `search` function above and see how `fmt` parameter is passed to the web service in order to make it return the results in `json` format.

Another thing that's worth mentioning is that if you open up the url provided by `get_release_url` function above it doesn't include the information you need. There is an `inc` parameter just like the `fmt` parameter that determines what extra data should be included in the response. Consult the [API docs](https://musicbrainz.org/doc/Development/JSON_Web_Service) to figure out the value for `inc` parameter that suggests it to include the track cound and disk count of the album (release).

There are a ton of other information freely available on the music brainz database. You can browse the [API docs](https://musicbrainz.org/doc/Development/JSON_Web_Service) and use it to extract more information. Feel free to do that.

In [8]:
# Your solution goes here
def getData(URL: str):
    return requests.get(
        URL,
        params={
            'fmt': 'json',
            'inc': 'discids'
        }
    ).json()
for value,items in top_5_albums.iterrows():
    try:
        URls =get_release_url( artist = items['Artist'],  title = items['Title'])
    except:
        print("first value not available")
    a = (URls)
    print(a)
TrackFrame= pd.DataFrame(columns = ['Track-Counts', 'Discs'])
for value,items in top_5_albums.iterrows():
        URls =get_release_url( artist = items['Artist'],  title = items['Title'])
        result = getData(a)['media']
        for read in result:
            track = (read['track-count'])
            disc =  len(read['discs'])
            TrackFrame = TrackFrame.append({'Track-Counts':track,'Discs':disc},ignore_index=True)
MainTable= pd.concat([TrackFrame,top_5_albums],axis = 1)
display(MainTable)
            
    
    
    
    

None
http://musicbrainz.org/ws/2/release/bdf93dcf-5c64-4ad2-80e8-0779a03ec218/
http://musicbrainz.org/ws/2/release/061e483c-7d6c-4864-ab94-546207952040/
http://musicbrainz.org/ws/2/release/0952fb25-b541-417a-948a-e821e2d51ad8/
http://musicbrainz.org/ws/2/release/76086324-51a9-4bf8-bf6f-93c5795839ec/


,Track-Counts,Discs,Title,Artist,Rank
0,13,0,Love Yourself: Tear,BTS,1
1,13,0,beerbongs & bentleys,Post Malone,2
2,13,0,Harder Than Ever,Lil Baby,3
3,13,0,And Justice For None,Five Finger Death Punch,4
4,13,0,Invasion Of Privacy,Cardi B,5
